In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pyarrow

In this notebook, I filter for morning rideshares in 2019 Q4 and 2022Q4 i.e., the first and last full quarters in pre and post pandemic (arguably). I later used this to create the visual in 'ManhattanCommutes_Pandemic_Before_and_after.twb'.

In [7]:
rideshares = pd.read_parquet(r'C:\Users\EvanSpiller\Documents\DS BootCamp\Capstone\data\HVFHV_Compressed\Combined\full_compressed_HVFHV_2019_2020.parquet')

In [8]:
#filtering for morning rides in 2019
#I decided on hours 7-10 by looking at histogram in an exploratory phase.
rideshares2019Q4morning = rideshares.loc[
    (
       ((rideshares['Month'] == 10) |(rideshares['Month'] == 11) | (rideshares['Month'] == 12)) &
    ((rideshares['Year'] == 2019) | (rideshares['Year'] == 2022))
     & ((rideshares['Hour']>=7) & (rideshares['Hour']<=10))
    )
]


In [9]:
#filtering for morning rides in 2022
#I decided on hours 7-10 by looking at histogram in an exploratory phase.

rideshares = pd.read_parquet(r'C:\Users\EvanSpiller\Documents\DS BootCamp\Capstone\data\HVFHV_Compressed\Combined\full_compressed_HVFHV_2022_2023.parquet')
rideshares2022Q4morning = rideshares.loc[
    (
       ((rideshares['Month'] == 10) |(rideshares['Month'] == 11) | (rideshares['Month'] == 12)) &
    ((rideshares['Year'] == 2019) | (rideshares['Year'] == 2022))
     & ((rideshares['Hour']>=7) & (rideshares['Hour']<=10))
    )
]


In [10]:
#concatenated both results
morningcommutesq420192022 = pd.concat([rideshares2022Q4morning, rideshares2019Q4morning], axis=0)


In [11]:
#adding in info about the zones e.g. names and boroughs
zones = pd.read_csv(r'C:\Users\EvanSpiller\Documents\DS BootCamp\Capstone\data\taxi+_zone_lookup.csv')

In [12]:
morningcommutesq420192022 = pd.merge(morningcommutesq420192022, zones, left_on='DOLocationID', right_on='LocationID', how='inner')


In [13]:
#filtering for just Manhattan
morningcommutesq420192022manhattan = morningcommutesq420192022.loc[morningcommutesq420192022['Borough']=='Manhattan',:]

In [14]:
morningcommutesq420192022manhattan

,PULocationID,DOLocationID,Year,Month,Day,Hour,trip_miles_sum,congestion_surcharge_sum,tips_sum,driver_pay_sum,trips,LocationID,Borough,Zone,service_zone
77333,2.0,230.0,2022,10,9,7,18.336,2.75,15.17,36.80,1,230,Manhattan,Times Sq/Theatre District,Yellow Zone
77334,3.0,230.0,2022,10,3,9,33.885,5.50,0.00,105.90,2,230,Manhattan,Times Sq/Theatre District,Yellow Zone
77335,3.0,230.0,2022,10,3,10,13.640,2.75,0.00,36.97,1,230,Manhattan,Times Sq/Theatre District,Yellow Zone
77336,3.0,230.0,2022,10,4,9,15.058,2.75,0.00,57.86,1,230,Manhattan,Times Sq/Theatre District,Yellow Zone
77337,3.0,230.0,2022,10,5,7,16.390,2.75,0.00,39.19,1,230,Manhattan,Times Sq/Theatre District,Yellow Zone
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6945731,181.0,105.0,2019,10,4,7,3.526,0.00,6.92,14.56,1,105,Manhattan,Governor's Island/Ellis Island/Liberty Island,Yellow Zone
6945732,195.0,105.0,2019,10,31,10,3.300,0.00,0.00,13.77,1,105,Manhattan,Governor's Island/Ellis Island/Liberty Island,Yellow Zone
6945733,188.0,105.0,2019,11,8,8,9.026,2.75,5.55,29.32,1,105,Manhattan,Governor's Island/Ellis Island/Liberty Island,Yellow Zone
6945734,132.0,105.0,2019,12,6,8,26.240,0.00,0.00,56.95,1,105,Manhattan,Governor's Island/Ellis Island/Liberty Island,Yellow Zone


Grouping by pickup location -- the ultimate goal is to see which pickup locations have more/fewer commutes when you compare quarters

In [15]:
groupby = morningcommutesq420192022manhattan.groupby(['PULocationID','Year'])['trips'].sum()

In [16]:
groupby = groupby.unstack()

In [17]:
groupby = groupby.reset_index()

In [18]:
groupby

Year,PULocationID,2019,2022
0,1.0,265.0,NaN
1,2.0,NaN,5.0
2,3.0,1657.0,1890.0
3,4.0,47118.0,23441.0
4,5.0,64.0,94.0
...,...,...,...
258,261.0,18441.0,18894.0
259,262.0,74892.0,45646.0
260,263.0,67161.0,47881.0
261,264.0,1.0,NaN


Sending to CSV so I can graph in Tableau.

In [51]:
groupby.to_csv(r'C:\Users\EvanSpiller\Documents\DS BootCamp\Capstone\data\morningcommutestomanhattanq4comparison.csv', index=False)